#Conectar con la API de Spotify

In [1]:
import requests 
import json
import networkx as nx
import matplotlib.pyplot as plt 
import time
from tqdm import tqdm
import numpy as np
import musicbrainzngs as mb
from os import environ
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from typing import List, Dict
import pickle


In [2]:
CLIENT_ID = '14c49fbcea5a4439bfa94aa591089353'
CLIENT_SECRET = '710e61b1ca974449a6c7a23c50146cca'

#CLIENT_ID = '3a283f4bd64241238d019491e4aedca3'
#CLIENT_SECRET = 'aee8c6c05d0a42c487c0069b7bd075a8'

# POST donde le pasamos las clave de la app
response = requests.post('https://accounts.spotify.com/api/token', data = {'grant_type': 'client_credentials', 'client_id': CLIENT_ID, 'client_secret': CLIENT_SECRET})
# Guardamos el bearer token para usarlo en las peticiones de la API
access_token = response.json()['access_token']
headers = {'Authorization': 'Bearer {}'.format(access_token), 'Accept': 'application/json', 'Content-Type': 'application/json'}

# OAuth_Client_ID = 'SObZ1oUfqqaJS5TLsZ2Ohy4LdqMa3Mpr'
# OAuth_Client_Secret = 'kUXJwU5WqBa5C7CkjCUITfdL6NQMd-4m'
#Identificación

mb.set_useragent('redes_complejas', 0.1) 

In [3]:
def conseguir_id_y_genero(artista):
    """
    Recibe
        artista: Nombre del artista del cual se quiere su ID
    Devuelve:
        id: La ID del artista y el/los generos del artista
    """
    # End point para obtener los audio features. Esto se saca de la referencia de la documentación
    url = 'https://api.spotify.com/v1/search'

    # Búsqueda. Acá sí hay que pasarselo como parámetros
    params = {'q': f"{artista}", 'type': 'artist', 'limit': '2'}

    # En este caso no lleva ningún parámetro, el id de la canción va directamente en el url
    response = requests.get(url, params = params, headers = headers)

    # Vemos el json de la respuesta
    json_data = response.json()
    genero = json_data['artists']['items'][0]['genres']
    
    # Con la siguiente lista conseguimos el ID del primer resultado de la busqueda (creo)
    id = json_data["artists"]["items"][0]["id"]

    return (id,genero)

In [9]:
def conseguir_cuadrivector(artista): 
    """
    Recibe
        nombre: artista 
    Devuelve:
        lista de cuadrivectores: [(artista,artistas_colaborador1,idcancion,fecha), (devueltaigual),..., ]
    """
    
    id = conseguir_id_y_genero(artista)[0]
    canciones = []
    #pido la data de los albumes del artista
    url = f'https://api.spotify.com/v1/artists/{id}/albums'
    response = requests.get(url, params = {}, headers = headers)
    json_data = response.json()
    items = json_data['items']
    #me guardo todos sus albumes
    
    

    canciones_junto_a_id = []
    
    
    for indice_album,item in enumerate(items):
        año_album = items[indice_album]['release_date']
        nombre_album = items[indice_album]['id']
        
        url = f'https://api.spotify.com/v1/albums/{nombre_album}/tracks'
        response = requests.get(url, params = {}, headers = headers)
        data_albumes = response.json()
        
        #itero sobre todas las canciones de cada album y la guardo en la lista de canciones
        data_canciones = data_albumes['items']
        
        for index_cancion in range(len(data_canciones)):
            
            song = data_canciones[index_cancion]['id']
            
            url = f' https://api.spotify.com/v1/tracks/{song}'
            response = requests.get(url, params = {}, headers = headers)
            json_data = response.json()
            
            for index_colaborador in range(len(json_data['artists'])):
                
                colaborador = json_data['artists'][index_colaborador]['name'] 

                canciones_junto_a_id.append((artista,colaborador,song,año_album))
                if canciones_junto_a_id[-1][0] == canciones_junto_a_id[-1][1]: #   AGREGAMOS ESTO PARA quitar autoenlaces
                    canciones_junto_a_id.pop(-1)
    
    
    return canciones_junto_a_id

In [4]:
def crear_enlaces(nombre_artista,colaboraciones):
    # Crea una lista con todos los enlaces
    enlaces = [(nombre_artista,colaboracion) for colaboracion in colaboraciones]

    return enlaces

In [5]:
def conseguir_pais_y_genero(nombre_artista): #AGREGAR ACA SI QUEREMOS ALGUN OTRO ATRIBUTO
    # Metemos a Argentina en la lista de provincias por las dudas ya que hay artistas que lo tienen
    lista_provincias = ["Argentina","Buenos Aires", "Catamarca", "Chaco", "Chubut", "Córdoba", "Corrientes", "Entre Ríos", "Formosa", "Jujuy", "La Pampa", "La Rioja", "Mendoza", "Misiones", "Neuquén", "Río Negro", "Salta", "San Juan", "San Luis", "Santa Cruz", "Santa Fe", "Santiago del Estero", "Tierra del Fuego", "Tucumán"]
    dic_atributos = {}
    
    data = mb.search_artists(artist = nombre_artista, limit= 1)

    if data["artist-count"] != 0: #SI ENCUENTRA EL ARTISTA EN MUSIC BRANDZ
        if "country" in data["artist-list"][0]:
            pais = data["artist-list"][0]["country"]
        # Comprobamos las provincias si no está el país
        elif "begin-area" in data["artist-list"][0]:
            provincia = data["artist-list"][0]["begin-area"]["name"]
            if provincia in lista_provincias:
                pais = "AR"
        elif "area" in data["artist-list"][0]:
            provincia = data["artist-list"][0]["area"]["name"]
            if provincia in lista_provincias:
                pais = "AR"

        dic_atributos["pais"] = pais
            
        if "gender" in data["artist-list"][0]:
            genero = data["artist-list"][0]["gender"]
            dic_atributos["genero"] = genero
    
    return dic_atributos

In [5]:
conseguir_pais_y_genero(artistas_argentinos[19])

NameError: name 'artistas_argentinos' is not defined

In [11]:
artistas_argentinos[1]

'Attaque 77'

In [26]:

# https://github.com/plamere/spotipy/blob/master/examples/playlist_tracks.py


# Authenticate to Spotify
def authenticate(cliend_id: str, client_secret: str) -> spotipy.client.Spotify:
    sp = spotipy.Spotify(
        client_credentials_manager=SpotifyClientCredentials(
            client_id=cliend_id,
            client_secret=client_secret
        )
    )

    return sp

autentificador = authenticate('14c49fbcea5a4439bfa94aa591089353','710e61b1ca974449a6c7a23c50146cca')
# Number of tracks available in the playlist
def get_pl_length(sp: spotipy.client.Spotify, pl_uri: str) -> int:
    return sp.playlist_tracks(
        pl_uri,
        offset=0,
        fields="total"
    )["total"]
    
    
def get_tracks_artist_info(sp: spotipy.client.Spotify, pl_uri: str) -> List[List[Dict]]:
    artists_info = list()
    # Start retrieving tracks from the beginning of the playlist
    offset = 0
    pl_length = get_pl_length(sp, pl_uri)

    # Playlist track retrieval only fetches 100 tracks at a time, hence\
    # the loop to keep retrieving until we reach the end of the playlist
    while offset != pl_length:
        # Get the next batch of tracks
        pl_tracks = sp.playlist_tracks(
            pl_uri,
            offset=offset,
            fields="items.track"
        )

        # Get the list with the info about the artists of each track from the\
        # latest batch and append it to the running list
        
        [artists_info.append(pl_item["track"]["artists"]) for pl_item in pl_tracks["items"]]

        # Update the offset
        offset += len(pl_tracks["items"])

    return artists_info



In [ ]:
artistas_argentinos = []

data_artistas = get_tracks_artist_info(autentificador,"https://open.spotify.com/playlist/6oF4i9FMoWGfvwUUI7IpgH?si=604e37fad5304c52")
for artista in data_artistas:
    nombre_artista = artista[0]["name"]
    pais_y_genero_artista = conseguir_pais_y_genero(nombre_artista)
    try:
        if pais_y_genero_artista["pais"] == "AR" and nombre_artista not in artistas_argentinos:
            artistas_argentinos.append(nombre_artista)
    except:
        print()

In [10]:
print(artistas_argentinos)
print(len(artistas_argentinos)) ##LA PLAYLIST TIENE 50 ARTISTAS, SOLO 3 NO TIENEN "COUNTRY" o no son AR.

['Charly García', 'Soda Stereo', 'Los Abuelos De La Nada', 'Los Enanitos Verdes', 'Virus', 'WOS', 'David Lebón', 'Los Piojos', 'Divididos', 'Gustavo Cerati', 'Conociendo Rusia', 'Fito Paez', 'Las Pastillas del Abuelo', 'Estelares', 'Vicentico', 'Luis Alberto Spinetta', 'Indio Solari', 'Sumo', 'Eruca Sativa', 'Emmanuel Horvilleur', 'Dante Spinetta', 'Skay Beilinson', 'Callejeros', 'Attaque 77', 'La Franela', 'Intoxicados', 'Los Caballeros De La Quema', 'Los Fabulosos Cadillacs', 'Tan Bionica', 'Bersuit Vergarabat', 'Las Pelotas', 'La Beriso', 'La Renga', 'Turf', 'Andrés Calamaro', 'Los Auténticos Decadentes', 'Gustavo Cordera', 'Serú Girán', 'Pappo', 'Dread Mar I', 'Babasónicos', 'Ivan Noble', 'Guasones']
43


In [ ]:
#URLS DE PLAYLIST CON ARTISTAS ARGENTINOS:
#rock nacional, folcrore argentino, los 90 argentina.

artistas_argentinos = []
artistas_quiza_argentinos = []

playlists = ["https://open.spotify.com/playlist/37i9dQZF1DX2v4h8QERQPb?si=f0244631d0084cb6","https://open.spotify.com/playlist/4BfzTfnImLBKkhYZwgsRMO?si=562120a745954c53","https://open.spotify.com/playlist/6GxBZn0yu0miclv2bhImBd?si=c513cd4ed3784604", "https://open.spotify.com/playlist/37i9dQZF1DX8KUmmjlkOod?si=fbfbb389e3034a54","https://open.spotify.com/playlist/6oF4i9FMoWGfvwUUI7IpgH?si=604e37fad5304c52"] 

for playlist in playlists:
    data_artistas = get_tracks_artist_info(autentificador,playlist)
    
    
    for artista in data_artistas:
        
        nombre_artista = artista[0]["name"]
        # print(nombre_artista)
        
        pais_y_genero_artista = conseguir_pais_y_genero(nombre_artista)
        
        try:
            if pais_y_genero_artista["pais"] == "AR" and nombre_artista not in artistas_argentinos:
                artistas_argentinos.append(nombre_artista)
        except:
            if nombre_artista not in artistas_quiza_argentinos:
            
                artistas_quiza_argentinos.append(nombre_artista)

La siguiente celda es para mergear la que tira last fm con la de Nico

In [13]:
import requests
from bs4 import BeautifulSoup as BS 
import time
import numpy as np
response = requests.get("https://www.last.fm/es/tag/argentina/artists")

# Vemos el contenido que nos devolvió
print(response.status_code)
# Creamos una lista para guardar todas las páginas de lastfm de artistas argentinos.
urls = []
for i in range(1,49):
    urls.append(f'https://www.last.fm/es/tag/argentina/artists?page={i}')

# Nos guardamos los artistas que encontramos en lastfm
artistas = []
# Despues guardamos los que sabemos que son argentinos por musicbrandz
artistas_argentinos = []
# Guardamos los artistas que no cumplan esto
artistas_quiza_argentinos = []
for url in urls:
    response = requests.get(url)
    soup = BS(response.content)
    elements = soup.findAll(attrs = {"class": "big-artist-list-title"})
    for element in elements:
        artistas.append(element.a.text)

for artista in artistas:  
        pais_y_genero_artista = conseguir_pais_y_genero(artista)
        time.sleep(1)
        try:
            if pais_y_genero_artista["pais"] == "AR" and artista not in artistas_argentinos:
                artistas_argentinos.append(artista)
        except:
            if artista not in artistas_quiza_argentinos:
                artistas_quiza_argentinos.append(artista)

artistas_argentinos_nico = ['Bardero$', 'WOS', 'YSY A', 'Nicki Nicole', 'Dani Ribba', 'Dillom', 'Bizarrap', 'Trueno', 'Homer El Mero Mero', 'ACRU', 'Franky Style', 'Kris Alaniz', 'Kodigo', 'Fianru', 'Lil Troca', 'Veeyam', 'Marcianos Crew', 'KHEA', 'Frijo', 'FMK', 'Lucho SSJ', 'Pekeño 77', 'Chaqueño Palavecino', 'Soledad', 'Horacio Guarany', 'Los Huayra', 'Facundo Toro', 'Los Nombradores del Alba', 'Nacho Prado Y Daniel Campos', 'Sandra Luna', 'Los Chalchaleros', 'Jorge Rojas', 'Daniel Toro', 'Los Nocheros', 'Luciano Pereyra', 'Roberto Carabajal', 'Abel Pintos', 'Peteco Carabajal', 'Roxana Carabajal', 'Mercedes Sosa', 'Raly Barrionuevo', 'Andrés Calamaro', 'Soda Stereo', 'Los Fabulosos Cadillacs', 'Los Auténticos Decadentes', 'Los Enanitos Verdes', 'Luis Alberto Spinetta', 'Gustavo Cerati', 'Fito Paez', 'Los Pericos', 'Patricio Rey y sus Redonditos de Ricota', 'Vilma Palma e Vampiros', 'Los Caballeros De La Quema', 'Attaque 77', 'Diego Torres', 'Gilda', 'Grupo Sombras', 'El Soldado', 'Fabiana Cantilo', 'Los Piojos', 'Bersuit Vergarabat', 'La Portuaria', 'Los Ladrones Sueltos', 'Memphis La Blusera', 'La Mosca Tse-Tse', 'Divididos', 'Sergio Denis', 'León Gieco', 'Charly García', 'Rafaga', 'Illya Kuryaki & The Valderramas', 'La Zimbabwe', 'Rata Blanca', 'Manuel Wirzt', 'Alcides', 'Los Abuelos De La Nada', 'Virus', 'David Lebón', 'Conociendo Rusia', 'Las Pastillas del Abuelo', 'Estelares', 'Vicentico', 'Indio Solari', 'Sumo', 'Eruca Sativa', 'Emmanuel Horvilleur', 'Dante Spinetta', 'Skay Beilinson', 'Callejeros', 'La Franela', 'Intoxicados', 'Tan Bionica', 'Las Pelotas', 'La Beriso', 'La Renga', 'Turf', 'Gustavo Cordera', 'Serú Girán', 'Pappo', 'Dread Mar I', 'Babasónicos', 'Ivan Noble', 'Guasones']
artistas_quiza_argentinos_nico = ['Duki', 'El Triangulo Estudio', 'Ronpe 99', 'G Sony', 'Slim Dee', 'Ronpe 99', 'Flaco Vazquez', 'Slim Dee', 'Brapis', 'Wolty', 'Dtoke', 'Iacho', 'Rapstinencia', 'La Joaqui', 'Urbanse', 'Urbanse', 'Wolty', 'Zica', 'Ronpe 99', 'Zica', 'Ronpe 99', 'Duki', 'Duki', 'Khea;Omar Varela', 'LIT killah', 'Duki', 'Duki', 'Duki', 'LIT killah', 'Duki', 'Neo Pistea', 'Seven Kayne', 'Duki', 'LIT killah', 'Duki', 'Duki', 'Duki', 'Midel', 'Seven Kayne', 'Ecko', 'Ecko', 'Luck Ra;Seven Kayne', 'Cumbia Trampa', 'Duki', 'Duki', 'Zanto', 'Neo Pistea', 'Felp 22', 'LIT killah', 'Seven Kayne', 'Neo Pistea', 'Duki', 'Duki', 'Duki', 'Duki', 'Duki', 'Duki', 'Antezanax 247', 'Campedrinos', 'Sergio Galleguillo', 'Tamara Castro', 'Los Rojas', 'Campedrinos', 'Campedrinos', 'Campedrinos', 'Campedrinos', 'Campedrinos', 'Daniela Toro', 'Sergio Galleguillo Y Los Amigos', 'Campedrinos', 'Los Carabajal', 'Tamara Castro', 'Campedrinos', 'Tamara Castro', 'Sergio Galleguillo', 'Sergio Galleguillo Y Los Amigos', 'Amboé', 'Campedrinos', 'Campedrinos', 'Amar Azul']

#mergeo lo de nico con lo de ari

artistas_argentinos = artistas_argentinos+artistas_argentinos_nico
artistas_quiza_argentinos = artistas_quiza_argentinos + artistas_quiza_argentinos_nico

200


In [21]:
#pickle.dump(artistas_argentinos, open('lista artistas argentinos', 'wb'))
#pickle.dump(artistas_quiza_argentinos, open('lista artistas quiza argentinos', 'wb'))


In [6]:
with open('lista artistas argentinos', 'rb') as f:
    artistas_argentinos = pickle.load(f)

with open('lista artistas quiza argentinos', 'rb') as f:
    artistas_quiza_argentinos = pickle.load(f)

In [ ]:
# Defino una lista con todos los artistas

# Defino una lista para guardar los enlaces
cuadrivectores_artistas_argentinos = []

for artista in artistas_argentinos:
    lista_cuadrivectores = conseguir_cuadrivector(artista)
    for vector in lista_cuadrivectores:
        nombre_colaborador = vector[1]
        pais_y_genero_colaborador = conseguir_pais_y_genero(nombre_colaborador)
        time.sleep(2)
        try:
            if pais_y_genero_colaborador["pais"] == "AR":
                cuadrivectores_artistas_argentinos.append(vector)
        except:
            pass

In [7]:
client_ids = ['3a283f4bd64241238d019491e4aedca3',"5b52471dd4894ebbbbd0a5b63e33cf63","3a533c6fd3434a29a09896712d5c19bd","14c49fbcea5a4439bfa94aa591089353","8b3a74b411c14db295be20c03a164241","62dad7314da84b22b885bab72e7f2d60","c8a66e35cf214da89054ba73d3c6fa38","78d1a19d9dbd4672a6e404648a02a002", "90eac8c23ced4e508e3c24aa6963e9dd"]
client_secrets = ['aee8c6c05d0a42c487c0069b7bd075a8',"62730cfc96a54bc298c2587228ac147e", "65f9ad113dd54f6091cee7aa5498568b","710e61b1ca974449a6c7a23c50146cca","9078850544c14866b525ab61aa7dfdf2","7a346376cdaf43948b16b436ca62c8a9","88e65321317744a6bad17c0522a63756", "dfebc28a789d44b89ad0fdce779af0e2", "1c8c10c2b9d94934b733b26fd06a3783"]

In [12]:
i = 0
index_aut = 0

cuadrivectores_artistas_argentinos = []
cuadrivectores_artistas_quiza_argentinos = []
artistas_argentinos = ["Alberto Rojo"]

#NOS AUTENTIFICAMOS PRIMERO
response = requests.post('https://accounts.spotify.com/api/token', data = {'grant_type': 'client_credentials', 'client_id': client_ids[0], 'client_secret': client_secrets[0]})
access_token = response.json()['access_token']
headers = {'Authorization': 'Bearer {}'.format(access_token), 'Accept': 'application/json', 'Content-Type': 'application/json'} 


#RECORREMOS LA LISTA DE ARTISTAS HASTA TERMINARLA: i ES EL INDICE QUE RECORRE
while i < 1:#len(artistas_argentinos):
    
    artista = artistas_argentinos[i] #DEFINO EL ARTISTA
    print(i)
    
    try:
        
        lista_cuadrivectores = conseguir_cuadrivector(artista)  # CONSIGO LOS CUADRIVECTORES DEL ARTISTA
        for cuadrivector in lista_cuadrivectores:
            nombre_colaborador = cuadrivector[1]
            try:
                pais_colaborador = conseguir_pais_y_genero(nombre_colaborador)["pais"]
                if pais_colaborador == "AR":
                    cuadrivectores_artistas_argentinos.append(lista_cuadrivectores)
                    artistas_argentinos.append(nombre_colaborador)
            except Exception as e:
                print(f"{nombre_colaborador} no tiene pais, lo meto en quiza argentino") 
                artistas_quiza_argentinos.append(nombre_colaborador)

        
        i += 1 # Pasamos al siguiente artista
                
    except Exception as e: 
        
        print(e)
        # Si el error es de indice, es que no encontró al artista en spotify, en ese caso, pasamos al siguiente artista.
        if type(e) == IndexError:
            i += 1
        print(artista)
        index_aut += 1
        if index_aut == len(client_ids):
            index_aut = 0
        response = requests.post('https://accounts.spotify.com/api/token', data = {'grant_type': 'client_credentials', 'client_id': client_ids[index_aut], 'client_secret': client_secrets[index_aut]})
        access_token = response.json()['access_token']
        headers = {'Authorization': 'Bearer {}'.format(access_token), 'Accept': 'application/json', 'Content-Type': 'application/json'} 

0
Studio ensemble no tiene pais, lo meto en quiza argentino
Studio ensemble no tiene pais, lo meto en quiza argentino
Studio ensemble no tiene pais, lo meto en quiza argentino
Studio ensemble no tiene pais, lo meto en quiza argentino
Studio ensemble no tiene pais, lo meto en quiza argentino
Pyotr Ilyich Tchaikovsky no tiene pais, lo meto en quiza argentino
Copyright Control no tiene pais, lo meto en quiza argentino
Copyright Control no tiene pais, lo meto en quiza argentino
Martín Telechanski no tiene pais, lo meto en quiza argentino
Mariana Baggio no tiene pais, lo meto en quiza argentino
Rita Del Prado no tiene pais, lo meto en quiza argentino
Martín Telechanski no tiene pais, lo meto en quiza argentino
Martín Telechanski no tiene pais, lo meto en quiza argentino
Sandra Corizzo no tiene pais, lo meto en quiza argentino
Sandra Corizzo no tiene pais, lo meto en quiza argentino
Miryam Quiñones no tiene pais, lo meto en quiza argentino
Miryam Quiñones no tiene pais, lo meto en quiza arge

KeyboardInterrupt: 

In [11]:
conseguir_pais_y_genero("Avicii")

{'pais': 'SE', 'genero': 'male'}

Guardo las listas actualizadas al 31/10

In [17]:
pickle.dump(artistas_argentinos, open('lista artistas argentinos', 'wb'))
pickle.dump(artistas_quiza_argentinos, open('lista artistas quiza argentinos', 'wb'))

Guardo los cuadrivectores también

In [27]:
pickle.dump(cuadrivectores_artistas_argentinos, open('cuadrivectores argentinos', 'wb'))
# filename = f"red_artistas_argentinos_lastfm_{i}"
# np.savez(filename+".npz", cuadrivectores = cuadrivectores_artistas_argentinos)

In [29]:
red_artistas= nx.Graph()
enlaces = []
for i in cuadrivectores_artistas_argentinos:
    for j in i:
        enlaces.append(j[0:2])

red_artistas.add_edges_from(enlaces)
#guardo para gephi y para pickle x las dudas
nx.write_gexf(red_artistas, "red_artistas_argentinos_0_199.gexf") 
nx.write_gpickle(red_artistas, "red_artistas_argentinos_0_199.gpickle")

Guardamos los artistas que quiza son argentinos

In [ ]:
# filename = f"red_artistas_quiza_argentinos_lastfm_{i}"
# np.savez(filename+".npz", cuadrivectores = cuadrivectores_artistas_argentinos)

Guardamos las listas de artistas

In [14]:
with open("red_artistas_argentinos_0_199.gpickle", "rb") as f:
    G = pickle.load(f)
.print(G)

Graph with 3394 nodes and 4536 edges


In [17]:
enlaces = list(G.edges())
for e in enlaces:
    if "Avicii" in e:
        print(e)

('La Mosca Tsé-Tsé', 'Avicii')


In [18]:
conseguir_cuadrivector("La Mosca Tsé-Tsé")

[('La Mosca Tsé-Tsé', 'Blackbear', '6t759ydOKpo7rwH408SMUP', '2019-07-09'),
 ('La Mosca Tsé-Tsé',
  'La Mosca Tse-Tse',
  '6t759ydOKpo7rwH408SMUP',
  '2019-07-09'),
 ('La Mosca Tsé-Tsé', 'Rey Ruiz', '6t759ydOKpo7rwH408SMUP', '2019-07-09'),
 ('La Mosca Tsé-Tsé', 'Blackbear', '1pOmspWOg63NCo8WOeRAxX', '2019-07-09'),
 ('La Mosca Tsé-Tsé',
  'La Mosca Tse-Tse',
  '1pOmspWOg63NCo8WOeRAxX',
  '2019-07-09'),
 ('La Mosca Tsé-Tsé', 'Rey Ruiz', '1pOmspWOg63NCo8WOeRAxX', '2019-07-09'),
 ('La Mosca Tsé-Tsé', 'Blackbear', '5dB6UrDCtj4OdBuvgOEI9o', '2019-07-09'),
 ('La Mosca Tsé-Tsé',
  'La Mosca Tse-Tse',
  '5dB6UrDCtj4OdBuvgOEI9o',
  '2019-07-09'),
 ('La Mosca Tsé-Tsé', 'Rey Ruiz', '5dB6UrDCtj4OdBuvgOEI9o', '2019-07-09'),
 ('La Mosca Tsé-Tsé', 'Blackbear', '2fKQ9Kg3x0k9K2FxadxyUM', '2019-07-09'),
 ('La Mosca Tsé-Tsé',
  'La Mosca Tse-Tse',
  '2fKQ9Kg3x0k9K2FxadxyUM',
  '2019-07-09'),
 ('La Mosca Tsé-Tsé', 'Rey Ruiz', '2fKQ9Kg3x0k9K2FxadxyUM', '2019-07-09'),
 ('La Mosca Tsé-Tsé', 'Blackbear', '2oPo

In [19]:
conseguir_pais_y_genero("Avicii")

{'pais': 'SE', 'genero': 'male'}